# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [3]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [4]:
load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
CHROMA_OPENAI_API_KEY=os.getenv("CHROMA_OPENAI_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")

### VectorDB Instance

In [5]:
chroma_client = chromadb.PersistentClient(path="./starter/chromadb")

### Collection

In [6]:
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_base="https://openai.vocareum.com/v1",
    api_key=OPENAI_API_KEY,
    model_name='text-embedding-3-large'
)

In [7]:
# Create a collection
collection = chroma_client.get_or_create_collection(
   name="udaplay",
   embedding_function=embedding_fn
)

### Add documents

In [42]:
# Make sure you have a directory "project/starter/games"
data_dir = "starter/games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [9]:
query = "A game where you race cars"

results = collection.query(
        query_texts=[query],
        n_results=5,
)

for i, doc in enumerate(results['documents'][0]):
        print(f"Result {i+1}:")
        print(doc)
        print()

Result 1:
[PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.

Result 2:
[PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.

Result 3:
[Nintendo Switch] Mario Kart 8 Deluxe (2017) - An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.

Result 4:
[PlayStation 2] Grand Theft Auto: San Andreas (2004) - An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson.

Result 5:
[Xbox 360] Kinect Adventures! (2010) - A collection of mini-games designed to showcase the capabilities of the Kinect motion sensor.

